In [3]:
import re

In [13]:
# Extract telemetry bits

r = re.compile(r"UART: (\d+)")

test = "UART: 01"

tbits = r.match(test)
print(tbits.group(1))


01


In [54]:
# Function to claculate CRC8

def crc8(tbits):
    """
    cbits is str of len 80
    """

    # Ensure array is not all 0s
    if "1" not in tbits:
        return False

    xor = 0

    for i in range(10):
        xor ^= int(tbits[i*8:(i+1)*8], 2)
    
    return not bool(xor)

# Test
print(crc8("0" * 80))
print(crc8("1" * 80))
print(crc8("1" * 64 + "0" * 16))

False
True
True


In [64]:
def tbits_to_telem(tbits):
    "Convert bits to telemetry data"
    print(f"Temp C: {int(tbits[:8], 2)}")
    print(f"Volt V: {int(tbits[8:24], 2) / 100}")
    print(f"Curr A: {int(tbits[24:40], 2) / 100}")
    print(f"Co mAh: {int(tbits[40:56], 2)}")
    print(f"erpm  : {int(tbits[56:72], 2) / 100}")

In [59]:
tbits = "".join([r.match(line).group(1) for line in open("telem.txt").readlines()])

print(tbits)
print(len(tbits))

1110111101111011001001001110111101111011110111101111011001001110111101111011001001110111101111011110110011101111011110111101111011110111101111011110111101111011110111101111011001001110111101100111011110111101111011110111101100111011001110111101111011110110011101100100111011110111101100100111011110111101111011110110010010011101100100100111011110110011101111011110110010010011101111011110111101111011110110011101111011110111101100111011110111101100111011001001001001110111101100100111011110111101111011110111101100100111011110110010010011101111011001001001110111101100100111011110111101111011110111101111011001001110111101100100100111011110110010010011101111011001110111101111011110111101100111011110111101111011110110011101100100111011110111101111011110111101111011110111101111011110111101111011110110010011101100100111011110111101111011001001110111101111011001110111101111011001110111101111011110110011101111011110111101111011110111101111011110111101111011001110110010010011101100100100100111011001

In [68]:
# Crc on tbits

idxs = [i for i in range(len(tbits) - 80 + 1) if crc8(tbits[i:i+80])]
print(len(idxs))
# for i in range(len(tbits) - 80 + 1):
#     crc = crc8(tbits[i:i+80])
#     if crc:
#         print(i)


83


In [79]:
# Try these

idx = idxs[9]

print(tbits[idx: idx + 80])

tbits_to_telem(tbits[idx: idx + 80])


11101111011110111101111011110111101111011110111101111011110111101111011110111101
Temp C: 239
Volt V: 317.1
Curr A: 634.21
Co mAh: 61307
erpm  : 570.79


In [80]:
for idx in idxs:
    tbits_to_telem(tbits[idx: idx + 80])


Temp C: 189
Volt V: 613.07
Curr A: 556.11
Co mAh: 57079
erpm  : 460.13
Temp C: 123
Volt V: 570.79
Curr A: 456.87
Co mAh: 48623
erpm  : 264.91
Temp C: 217
Volt V: 570.78
Curr A: 303.26
Co mAh: 18727
erpm  : 264.68
Temp C: 179
Volt V: 486.2
Curr A: 606.52
Co mAh: 37454
erpm  : 529.37
Temp C: 206
Volt V: 200.83
Curr A: 401.67
Co mAh: 14793
erpm  : 101.07
Temp C: 156
Volt V: 401.67
Curr A: 147.98
Co mAh: 29586
erpm  : 202.14
Temp C: 217
Volt V: 570.78
Curr A: 306.53
Co mAh: 60573
erpm  : 613.07
Temp C: 179
Volt V: 486.2
Curr A: 613.07
Co mAh: 55611
erpm  : 570.79
Temp C: 103
Volt V: 317.05
Curr A: 570.79
Co mAh: 45687
erpm  : 486.23
Temp C: 239
Volt V: 317.1
Curr A: 634.21
Co mAh: 61307
erpm  : 570.79
Temp C: 222
Volt V: 634.21
Curr A: 613.07
Co mAh: 57079
erpm  : 486.23
Temp C: 189
Volt V: 613.07
Curr A: 570.79
Co mAh: 48623
erpm  : 317.1
Temp C: 123
Volt V: 570.79
Curr A: 486.23
Co mAh: 31710
erpm  : 634.21
Temp C: 247
Volt V: 486.23
Curr A: 317.1
Co mAh: 63421
erpm  : 613.07
Temp C: 239

# Telem 2

In [108]:
telem = """
UART: 11101
UART: 100
UART: 11100011
UART: 0
UART: 0
UART: 0
UART: 0
UART: 0
UART: 1110110
UART: 11011111
"""
print(telem)


UART: 11101
UART: 100
UART: 11100011
UART: 0
UART: 0
UART: 0
UART: 0
UART: 0
UART: 1110110
UART: 11011111



In [109]:
# Convert telem to 80 bits
r = re.compile(r"UART: (\d+)\n")
tbits = r.findall(telem)
print(tbits)
xor = 0
for t in tbits:
    xor ^= int(t, 2)
print(bin(xor))

['11101', '100', '11100011', '0', '0', '0', '0', '0', '1110110', '11011111']
0b1010011


In [110]:
# Print telem values

print(f"Temp C: {int(tbits[0], 2)}")
print(f"Volt V: {int(tbits[1] + tbits[2].rjust(8, '0'), 2) / 100}")
print(f"Curr A: {int(tbits[3] + tbits[4].rjust(8, '0'), 2) / 100}")
print(f"Co mAh: {int(tbits[5] + tbits[6].rjust(8, '0'), 2)}")
print(f"erpm  : {int(tbits[7] + tbits[8].rjust(8, '0'), 2) * 100}")

Temp C: 29
Volt V: 12.51
Curr A: 0.0
Co mAh: 0
erpm  : 11800


# Telem 3

Convert hex

In [8]:
telem = """
UART: 1e
UART: 3
UART: 63
UART: 0
UART: 0
UART: 0
UART: 1
UART: 1
UART: 5
UART: 87
"""
print(telem)


UART: 1e
UART: 3
UART: 63
UART: 0
UART: 0
UART: 0
UART: 1
UART: 1
UART: 5
UART: 87



In [9]:
# Convert telem to 80 bits
r = re.compile(r"UART: ([a-z0-9]+)\n")
tbits = r.findall(telem)
print(tbits)
xor = 0
for t in tbits:
    xor ^= int(t, 16)
print(bin(xor))

['1e', '3', '63', '0', '0', '0', '1', '1', '5', '87']
0b11111100


In [10]:
# Print telem values

print(f"Temp C: {int(tbits[0], 16)}")
print(f"Volt V: {int(tbits[1] + tbits[2].rjust(2, '0'), 16) / 100}")
print(f"Curr A: {int(tbits[3] + tbits[4].rjust(2, '0'), 16) / 100}")
print(f"Co mAh: {int(tbits[5] + tbits[6].rjust(2, '0'), 16)}")
print(f"erpm  : {int(tbits[7] + tbits[8].rjust(2, '0'), 16) * 100}")

Temp C: 30
Volt V: 8.67
Curr A: 0.0
Co mAh: 1
erpm  : 26100
